# カテゴリを予測
## 特徴量

## アルゴリズム
XGBoost

## 結果

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import gc
pd.set_option('display.max_columns', 200)

train_csv = pd.read_csv("../input/sf-crime/train.csv.zip")
test_csv  = pd.read_csv("../input/sf-crime/test.csv.zip")
sample_csv  = pd.read_csv("../input/sf-crime/sampleSubmission.csv.zip")
# col = ['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict','Resolution', 'Address', 'X', 'Y']
# train.shape = (878049, 9)
train_csv.head(3)

In [ ]:
#@function_define
# データ変換によってレコード数が誤って増減していないか確認する
def check(pre, new, columns=False):
    print("---------- check ----------")
    print("pre.shape = ", pre.shape)
    print("new.shape = ", new.shape)
    if columns:
        print("new.columns = ", new.columns)
    print("---------------------------")

# 外れ値の処理
X=-120.5, Y=90になっているレコードを外れ値として扱う。おそらく何かしらの不具合？

非外れ値側のPdDistrict毎のXとYの平均を計算し、外れ値側のX,Yをそれに置き換える

In [ ]:
# trainデータの外れ値の分割
train_raw = train_csv[train_csv["Y"] < 60]
train_raw_outlier = train_csv[train_csv["Y"] >= 60]

agg_pddistrict = train_raw[["PdDistrict", "X", "Y"]].groupby(
    ["PdDistrict"],
    as_index=False
).agg(
    {"X": np.mean, "Y": np.mean}
)
train_raw_outlier = train_raw_outlier.drop(["X", "Y"],axis=1).merge(agg_pddistrict, on="PdDistrict")
train_raw = pd.concat([train_raw, train_raw_outlier], axis=0)

check(train_csv, train_raw)
train_raw.head(3)

In [ ]:
# trainデータの外れ値の分割
test_raw = test_csv[test_csv["Y"] < 60]
test_raw_outlier = test_csv[test_csv["Y"] >= 60]

agg_pddistrict = test_raw[["PdDistrict", "X", "Y"]].groupby(
    ["PdDistrict"],
    as_index=False
).agg(
    {"X": np.mean, "Y": np.mean}
)
test_raw_outlier = test_raw_outlier.drop(["X", "Y"],axis=1).merge(agg_pddistrict, on="PdDistrict")
test_raw = pd.concat([test_raw, test_raw_outlier], axis=0)

all = pd.concat([train_raw, test_raw], axis=0)[["X", "Y"]]

check(test_csv, test_raw)
test_raw.head(3)

# マスタを作成

In [ ]:
category = train_raw["Category"].drop_duplicates().to_list()

# 特徴量生成に利用する関数の定義

In [ ]:
#@function_define
# 交差点で起きた事故、事件(=Addressに"/"が入っている)ものは、そうでないものと分ける

def add_intersection(data):
    data["Intersection"] = data["Address"].str.contains("/")
    return data

In [ ]:
#@function_define
# サンフランシスコで犯罪件数が多い通りの名前
street_name = ["SAN JOSE AV", "DOLORES ST", "VALENCIA ST", "MISSION ST"]
def add_street_flag(data):
    def get_st_name(x):
        st_flag = [(st in x) for st in street_name]
        if np.any(st_flag):
            return street_name[st_flag.index(True)]
        else:
            return ""
    data["Street_flag"] = data["Address"].map(get_st_name)
    return data

In [ ]:
#@function_define
# 日付から月、時刻、TimeGroup(朝昼晩区分)を追加する関数

def add_timegroup(data):
    data["Dates"] = pd.to_datetime(data["Dates"])
    data["Year"] = data["Dates"].dt.year
    data["Month"] = data["Dates"].dt.month
    data["Hour"] = data["Dates"].dt.hour
    # 定義は適当にPanasonicのスマート家電からhttps://panasonic.jp/pss/qa/answer167.html
    def func_cate(x):
        if  x >= 3 and x < 11:  # 朝は、3時から10時59分まで
            return "morning"
        elif x >= 11 and x < 18: # 昼は、11時から17時59分まで
            return "daytime"
        else:  # 夜は18時から26時59分まで
            return "evening"
    data['TimeGroup'] = data["Hour"].apply(func_cate)
    return data

In [ ]:
#@function_define
# 指定した文字列カラムを数字に変換する(LabelEncode)
from sklearn.preprocessing import LabelEncoder

def label_encode(data, columns):
    label_masters = {}
    for column in columns:
        le = LabelEncoder()
        le.fit(data[column])
        data[column+"_id"] = le.transform(data[column])
        label_masters.update({
            column: le.classes_
        })
    return data, label_masters

In [ ]:
#@function_define
# 周辺のカテゴリ別の犯罪件数の分布を付与する
def add_category_distribution(data, x_range, y_range, n_splits):
    # 指定した緯度経度の範囲をでn_split等分する
    x_div = (x_range[1]-x_range[0])/n_splits
    y_div = (y_range[1]-y_range[0])/n_splits
    def to_left(v):
        if type(v) is float:
            return v
        else:
            return v.left
    data["x_group"] = [to_left(x) for x in pd.cut(
        data["X"], np.arange(x_range[0], x_range[1]+x_div, x_div), right=False)]
    data["y_group"] = [to_left(y) for y in pd.cut(
        data["Y"], np.arange(y_range[0], y_range[1]+x_div, y_div), right=False)]
    
    # 各範囲毎の犯罪件数を集計する
    agg = data.copy()
    agg["count"] = 1
    agg = agg.groupby(
        [
            "Category",
            "x_group",
            "y_group"
        ],
        as_index = False
    ).agg(
        {"count": np.sum}
    )
    agg=agg.pivot(
        index=['x_group', 'y_group'], columns='Category', values='count'
    ).fillna(0).reset_index()
    
    # 各行ごとに犯罪の件数から割合に変換する
    agg_category = agg.drop(["x_group", "y_group"], axis=1)
    agg["sum"] = agg_category.sum(axis=1)
    for col in agg_category.columns:
        agg[col] = agg[col]/agg["sum"]
    agg = agg.drop("sum", axis=1)
    
    # dataの取り方によっては犯罪件数が0件のカテゴリが生じる
    # その場合は0で埋める
    for col in category:
        if not col in agg.columns:
            agg[col] = 0

    return pd.merge(data, agg, on=['x_group', 'y_group'],how='left'), agg

In [ ]:
# 上記関数を利用して特徴量生成
train = train_raw.copy()
train = add_intersection(train)
train = add_street_flag(train)
train = add_timegroup(train)
train, masters = label_encode(train, ["Street_flag", "Category"])
check(train_raw, train, columns = True)
train.head(3)

# trainデータを利用して学習する

## クロスバリデーション
StratifiedKFoldは指定したカテゴリーが各foldに均等に配分されるように分割してくれる。  
今回はCategory毎にレコード数に大きな差があり、普通のKFoldを使うとCategoryに偏りが生じてしまうため、StratifiedKFoldを利用している

In [ ]:
#@function_define
from sklearn.model_selection import StratifiedKFold
def cross_validation():
    skf = StratifiedKFold(n_splits=config.n_splits)
    return [v for v in skf.split(train, train["Category_id"])]

## XGBoost
xgboostライブラリのxgb.trainを利用する

In [ ]:
#@function_define
import xgboost as xgb

def run_single_xgboost(dtrain, dvalid, params):
    evals = [(dtrain, 'train'), (dvalid, 'eval')]
    
    # 学習過程を記録するための辞書
    evals_result = {}

    # モデルの学習の実行
    bst = xgb.train(
        params,
        dtrain,
        config.num_round,
        evals = evals,
        evals_result = evals_result,
    )

    return bst, evals_result

In [ ]:
#@function_define
# 学習結果の描画
import operator
import datetime

def visualize(bst, evals_result):
    train_metric = evals_result['train']['mlogloss']
    plt.plot(train_metric, label='train mlogloss')
    eval_metric = evals_result['eval']['mlogloss']
    plt.plot(eval_metric, label='eval mlogloss')
    plt.grid()
    plt.legend()
    plt.xlabel('rounds')
    plt.ylabel('mlogloss')
    plt.show()

## ハイパーパラメータのチューニング
Optunaを利用して、xgboostのハイパーパラメータをチューニングする。  
xgboostのパラメータについては[こちらのQiita記事](https://qiita.com/FJyusk56/items/0649f4362587261bd57a)を参考にした。  
optunaの使い方に関しては[こちらのQiita記事](https://qiita.com/mamorous3578/items/912f1a2be0e9da7e9140)を参考にした。

In [ ]:
#@function_define

def run_fold_xgboost(hyperparams):
    config.trial_count += 1
    eta = hyperparams.suggest_uniform('eta', 0.0, 1.0) # デフォルトは0.3
    gamma = hyperparams.suggest_uniform('gamma', 0.0, 100.0) # デフォルトは0
    n_splits_xy = hyperparams.suggest_int('n_splits_xy', 100, 1000)
    print("eta: ", eta)
    print("gamma: ", gamma)
    print("n_splits_xy: ", n_splits_xy)
    
    metric = []
    fold = 0
    for train_t_iloc, train_v_iloc in config.cross_validation:
        fold += 1
        print("---- fold " + str(fold) + "/" + str(config.n_splits) + " ----")
        train_t = train.iloc[train_t_iloc].copy()
        train_v = train.iloc[train_v_iloc].copy()
        
        # ■ 周辺のカテゴリ別の犯罪件数を付与する
        # validationデータにつけた犯罪数分布は本来予測して得られるものなので外し、代わりに学習用データから作成した犯罪件数を付与する。
        x_range = [min(all["X"]), max(all["X"])]
        y_range = [min(all["Y"]), max(all["Y"])]
        train_t, crime_hist = add_category_distribution(train_t, x_range, y_range, n_splits_xy)
        train_v, _ = add_category_distribution(train_v, x_range, y_range, n_splits_xy)
        train_v = train_v.drop(category, axis=1)
        train_v = train_v.merge(crime_hist, on=['x_group','y_group'], how="left")
        
        # ■ 必要なカラムだけ選択
        train_t_select = train_t[[
            'DayOfWeek',
            'PdDistrict',
            'Month',
            'TimeGroup',
            'Intersection',
            'Street_flag_id',
            'Category_id',
            *crime_hist.columns
        ]]
        train_v_select = train_v[[
            'DayOfWeek',
            'PdDistrict',
            'Month',
            'TimeGroup',
            'Intersection',
            'Street_flag_id',
            'Category_id',
            *crime_hist.columns
        ]]

        # ■ 決定木用にデータを処理
        # One-hot Encodingを行う
        train_dummies_t = pd.get_dummies(train_t_select)
        train_dummies_v = pd.get_dummies(train_v_select)
        # 特徴量と目的変数をxgboostのデータ構造に変換する
        dtrain = xgb.DMatrix(train_dummies_t.drop("Category_id", axis=1), label=train_dummies_t["Category_id"])
        dvalid = xgb.DMatrix(train_dummies_v.drop("Category_id", axis=1), label=train_dummies_v["Category_id"])
        
        gc.collect()
        
        # ■ 学習実行
        # パラメータの設定
        params = {
            # GPU
            # "tree_method": 'gpu_hist',
            # 'n_gpus': 1,
            # Learning Parameters
            "objective": 'multi:softprob',
            'num_class': 39,
            'eval_metric':'mlogloss',
            # Booster Parameters(ベイズ最適化の対象)
            # 固定値はライブラリで定められたデフォルト値をそのまま利用している。
            "eta": eta,
            "gamma": gamma,
            "max_depth": 6,
            "min_child_weight": 1,
            "max_delta_step": 0,
            "subsample": 1,
            "colsample_bytree": 1,
            "colsample_bylevel": 1,
            "lambda": 1,
            "alpha": 0
        }
        # 学習を実行
        bst, evals_result = run_single_xgboost(dtrain, dvalid, params)
        # 学習結果を保存
        config.storage_model(bst, fold)
        config.storage_importance(bst, fold)
        # 計算結果を可視化
        # visualize(bst, evals_result)
        
        metric.append({
            "train_metric": evals_result['train']['mlogloss'][-1],
            "eval_metric": evals_result['eval']['mlogloss'][-1]
        })
    
    print("---- fold finish ----")
    # 計算結果をファイル保存
    result = {
        "metric": metric,
        "train_ave": np.mean([m["train_metric"] for m in metric]),
        "eval_ave": np.mean([m["eval_metric"] for m in metric])
    }
    config.result.append(result)
    config.storage_result(result)
    
    return result["eval_ave"]

In [ ]:
#@function_define
# 学習全般に関するパラメータの管理
import os
import json
import pickle
class Config(object):
    def __init__(self):
        self.n_splits = 4   # Foldの数
        self.n_trials = 50   # Optuna(ハイパーパラメータチューニング)の試行回数
        self.num_round = 50   # xgboostの試行回数
        
        self.trial_count = 0
        self.cross_validation = []
        self.result = []
        
        # modelの計算結果をファイルとして保存するためのフォルダを作成
        dt_now = dt.datetime.now().strftime('%Y%m%d-%H%M%S')
        self.dir_name = "model_" + dt_now
        os.makedirs(self.dir_name)
    
    def storage_model(self, bst, fold):
        file_name = self.dir_name + "/model_trial-" + str(self.trial_count) + "_fold-" + str(fold) + ".bst"
        bst.save_model(file_name)

    def storage_importance(self, bst, fold):
        file_name = self.dir_name + "/importance_trial-" + str(self.trial_count) + "_fold-" + str(fold) + ".json"
        with open(file_name, 'w') as f:
            json.dump(bst.get_fscore(), f, indent=2)

    def storage_result(self, result):
        file_name = self.dir_name + "/result_trial-" + str(self.trial_count) + ".json"
        with open(file_name, 'w') as f:
            json.dump(result, f, indent=2)

    def storage_best_params(self, study):
        file_name = self.dir_name + "/best_params.json"
        with open(file_name, 'w') as f:
            json.dump({
                "best_params": study.best_params,
                "best_value": study.best_value
            }, f, indent=2)
    
    def storage_study(self, study):
        file_name = self.dir_name + "/study.pickle"
        with open(file_name, mode="wb") as f:
            pickle.dump(study, f)
        
    def storage_submission(self, submission):
        file_name = self.dir_name + "/submission.csv"
        submission.to_csv(file_name, index = False)

In [ ]:
import optuna

folder_name = None
if folder_name:
    print("----- load "+folder_name+" -----")
    with open("./"+folder_name+"/study.pickle", mode="rb") as f:
        study = pickle.load(f)
else:
    study = optuna.create_study()
    # hyperparamsの初期値を設定
    study.enqueue_trial({
        'eta': 0.14504447778508744,
        'gamma': 41.441405792041834,
        'n_splits_xy': 100
    })

# 学習パラメータを設定
config = Config()
config.n_splits = 3  # Foldの数
config.n_trials = 15   # Optuna(ハイパーパラメータチューニング)の試行回数
config.num_round = 50   # xgboostの試行回数

# クロスバリデーション
config.cross_validation = cross_validation()

# 最適なhyperparams探索を実行
study.optimize(run_fold_xgboost, n_trials=config.n_trials)

In [ ]:
config.storage_best_params(study)
config.storage_study(study)

# 最適パラメータの表示
print(study.best_params)
# 最小化された目的関数を表示
print(study.best_value)

# Submit

In [ ]:
# ■ 周辺のカテゴリ別の犯罪件数を付与する
x_range = [min(all["X"]), max(all["X"])]
y_range = [min(all["Y"]), max(all["Y"])]
train_all, _ = add_category_distribution(train, x_range, y_range, study.best_params["n_splits_xy"])

# ■ 必要なカラムだけ選択
train_all_select = train_all[[
    'DayOfWeek',
    'PdDistrict',
    'Month',
    'TimeGroup',
    'Intersection',
    'Street_flag_id',
    'Category_id',
    *_.columns
]]

# ■ 決定木用にデータを処理
# One-hot Encodingを行う
train_dummies_all = pd.get_dummies(train_all_select)

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain_all = xgb.DMatrix(train_dummies_all.drop("Category_id", axis=1), label=train_dummies_all["Category_id"])

gc.collect()
# ■ 学習実行
# パラメータの設定
config.num_round = 100   # xgboostの試行回数
params = {
    # Learning Parameters
    "objective": 'multi:softprob',
    'num_class': 39,
    'eval_metric':'mlogloss',
    # Booster Parameters(ベイズ最適化の対象)
    # 固定値はライブラリで定められたデフォルト値をそのまま利用している。
    "eta": study.best_params["eta"],
    "gamma": study.best_params["gamma"],
    "max_depth": 6,
    "min_child_weight": 1,
    "max_delta_step": 0,
    "subsample": 1,
    "colsample_bytree": 1,
    "colsample_bylevel": 1,
    "lambda": 1,
    "alpha": 0
}
# 学習を実行
bst, evals_result = run_single_xgboost(dtrain_all, dtrain_all, params)
# 学習結果を保存
config.trial_count = "finish"
config.storage_model(bst, 0)
config.storage_importance(bst, 0)
# 計算結果を可視化
visualize(bst, evals_result)

In [ ]:
test = test_raw.copy()
test = add_intersection(test)
test = add_street_flag(test)
test = add_timegroup(test)
test, _ = label_encode(test, ["Street_flag"])

tmp = test.copy()
tmp["Category"] = "-"
x_range = [min(all["X"]), max(all["X"])]
y_range = [min(all["Y"]), max(all["Y"])]
_, master_all = add_category_distribution(train_raw, x_range, y_range, study.best_params["n_splits_xy"])
tmp, _ = add_category_distribution(tmp, x_range, y_range, study.best_params["n_splits_xy"])
test = tmp.drop(["-"]+category, axis=1).merge(master_all, on=['x_group', 'y_group'], how="left")

test_select = test[[
    'DayOfWeek',
    'PdDistrict',
    'Month',
    'TimeGroup',
    'Street_flag',
    'Street_flag_id',
    'Intersection',
    *master_all.columns
]]


test_dummies = pd.get_dummies(test_select)
print(test.shape)
test_dummies.head(3)

In [ ]:
test_dummies_select = test_dummies[[*train_dummies_all.drop("Category_id", axis=1).columns]]

dtest = xgb.DMatrix(test_dummies_select)
pred_test = bst.predict(dtest)

# 提出用ファイルの作成
submisson = pd.DataFrame(pred_test, columns = masters["Category"])
submisson["Id"] = test_raw["Id"].to_list()

config.storage_submission(submisson)

submisson.head(3)